# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 24 2022 9:04AM,240378,10,0085798458,ISDIN Corporation,Released
1,Jun 24 2022 9:04AM,240378,10,0085798459,ISDIN Corporation,Released
2,Jun 24 2022 9:04AM,240378,10,0085798460,ISDIN Corporation,Released
3,Jun 24 2022 9:04AM,240378,10,0085798468,ISDIN Corporation,Released
4,Jun 24 2022 9:04AM,240378,10,0085798483,ISDIN Corporation,Released
5,Jun 24 2022 9:04AM,240378,10,0085798485,ISDIN Corporation,Released
6,Jun 24 2022 9:04AM,240378,10,0085798488,ISDIN Corporation,Released
7,Jun 24 2022 9:04AM,240378,10,0085798486,ISDIN Corporation,Released
8,Jun 24 2022 9:04AM,240378,10,0085798489,ISDIN Corporation,Released
9,Jun 24 2022 9:04AM,240378,10,0085798493,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,240377,Released,13
28,240378,Released,62
29,240379,Released,15
30,240381,Released,6
31,240382,Released,19


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240377,NaN,13.0
240378,NaN,62.0
240379,NaN,15.0
240381,NaN,6.0
240382,NaN,19.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240283,1.0,0.0
240285,0.0,1.0
240295,0.0,1.0
240311,1.0,1.0
240312,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240283,1,0
240285,0,1
240295,0,1
240311,1,1
240312,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240283,1,0
1,240285,0,1
2,240295,0,1
3,240311,1,1
4,240312,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240283,1,
1,240285,,1
2,240295,,1
3,240311,1,1
4,240312,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 24 2022 9:04AM,240378,10,ISDIN Corporation
62,Jun 24 2022 9:00AM,240382,12,Hush Hush
81,Jun 24 2022 8:57AM,240381,10,Emerginnova
87,Jun 24 2022 8:53AM,240379,10,ISDIN Corporation
102,Jun 24 2022 8:47AM,240377,10,Bio-PRF
115,Jun 24 2022 8:37AM,240374,15,"Person & Covey, Inc."
139,Jun 24 2022 8:32AM,240375,15,"Mizner Bioscience, LLC"
147,Jun 24 2022 8:25AM,240373,15,"Virtus Pharmaceuticals, LLC"
159,Jun 24 2022 8:15AM,240371,19,ACG North America LLC
160,Jun 24 2022 8:13AM,240370,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 24 2022 9:04AM,240378,10,ISDIN Corporation,,62
1,Jun 24 2022 9:00AM,240382,12,Hush Hush,,19
2,Jun 24 2022 8:57AM,240381,10,Emerginnova,,6
3,Jun 24 2022 8:53AM,240379,10,ISDIN Corporation,,15
4,Jun 24 2022 8:47AM,240377,10,Bio-PRF,,13
5,Jun 24 2022 8:37AM,240374,15,"Person & Covey, Inc.",,24
6,Jun 24 2022 8:32AM,240375,15,"Mizner Bioscience, LLC",,8
7,Jun 24 2022 8:25AM,240373,15,"Virtus Pharmaceuticals, LLC",,12
8,Jun 24 2022 8:15AM,240371,19,ACG North America LLC,,1
9,Jun 24 2022 8:13AM,240370,15,"Alliance Pharma, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 9:04AM,240378,10,ISDIN Corporation,62,
1,Jun 24 2022 9:00AM,240382,12,Hush Hush,19,
2,Jun 24 2022 8:57AM,240381,10,Emerginnova,6,
3,Jun 24 2022 8:53AM,240379,10,ISDIN Corporation,15,
4,Jun 24 2022 8:47AM,240377,10,Bio-PRF,13,
5,Jun 24 2022 8:37AM,240374,15,"Person & Covey, Inc.",24,
6,Jun 24 2022 8:32AM,240375,15,"Mizner Bioscience, LLC",8,
7,Jun 24 2022 8:25AM,240373,15,"Virtus Pharmaceuticals, LLC",12,
8,Jun 24 2022 8:15AM,240371,19,ACG North America LLC,1,
9,Jun 24 2022 8:13AM,240370,15,"Alliance Pharma, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 9:04AM,240378,10,ISDIN Corporation,62,
1,Jun 24 2022 9:00AM,240382,12,Hush Hush,19,
2,Jun 24 2022 8:57AM,240381,10,Emerginnova,6,
3,Jun 24 2022 8:53AM,240379,10,ISDIN Corporation,15,
4,Jun 24 2022 8:47AM,240377,10,Bio-PRF,13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 9:04AM,240378,10,ISDIN Corporation,62.0,NaN
1,Jun 24 2022 9:00AM,240382,12,Hush Hush,19.0,NaN
2,Jun 24 2022 8:57AM,240381,10,Emerginnova,6.0,NaN
3,Jun 24 2022 8:53AM,240379,10,ISDIN Corporation,15.0,NaN
4,Jun 24 2022 8:47AM,240377,10,Bio-PRF,13.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 9:04AM,240378,10,ISDIN Corporation,62.0,0.0
1,Jun 24 2022 9:00AM,240382,12,Hush Hush,19.0,0.0
2,Jun 24 2022 8:57AM,240381,10,Emerginnova,6.0,0.0
3,Jun 24 2022 8:53AM,240379,10,ISDIN Corporation,15.0,0.0
4,Jun 24 2022 8:47AM,240377,10,Bio-PRF,13.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1442142,97.0,1.0
12,961378,21.0,2.0
15,1442176,70.0,0.0
16,1442056,5.0,3.0
19,720991,3.0,0.0
20,720987,2.0,2.0
21,240295,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1442142,97.0,1.0
1,12,961378,21.0,2.0
2,15,1442176,70.0,0.0
3,16,1442056,5.0,3.0
4,19,720991,3.0,0.0
5,20,720987,2.0,2.0
6,21,240295,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,97.0,1.0
1,12,21.0,2.0
2,15,70.0,0.0
3,16,5.0,3.0
4,19,3.0,0.0
5,20,2.0,2.0
6,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,97.0
1,12,Released,21.0
2,15,Released,70.0
3,16,Released,5.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,1.0,2.0,0.0,3.0,0.0,2.0,0.0
Released,97.0,21.0,70.0,5.0,3.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,1.0,2.0,0.0,3.0,0.0,2.0,0.0
1,Released,97.0,21.0,70.0,5.0,3.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,1.0,2.0,0.0,3.0,0.0,2.0,0.0
1,Released,97.0,21.0,70.0,5.0,3.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()